<a href="https://colab.research.google.com/github/shantanu2383/FAMA-FRENCH-5-FACTORS-ST-REVERSAL-LT-REVERSAL-MOMENTUM/blob/main/01_20_FAMA_FRENCH_DAILY_MOMENTUM_FACTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps
from sqlite3 import connect
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

conn=connect(':memory:')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=af1f19f6ea138c28ee822f2bf21c1741f974f5e9a90d6b6f81605307a850ba48
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


# IMPORT CLEANED PORTFOLIO DATA FROM DAILY ST REVERSAL

Here we import the portfolio data from the daily ST reversal calculations, as these already have the relevant size portfolios.

In [ ]:

filepath="/content/gdrive/MyDrive/"
file="0120-daily_returns_portfolios.csv"

df=pd.read_csv(filepath + file)

In [ ]:
df.drop(columns=["Unnamed: 0", "index"], inplace=True)

In [ ]:
df

,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2,return_q30,return_q70,return_portfolio,median_size,size_portfolio
0,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687,-0.045778,0.027244,H,3032.882350,B
1,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568,-0.082924,-0.014383,M,2929.106730,B
2,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294,-0.072480,-0.005190,M,2947.142600,B
3,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803,-0.069319,-0.001807,M,2947.969625,B
4,A,2018-02-09 00:00:00,64.609962,1.029669,-0.081085,2017Q4,NaN,NYSE,321.83,20194.270530,-0.092648,-0.027115,M,2872.864853,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701852,ZYNE,2022-12-22 00:00:00,0.540000,0.929432,0.096226,2022Q3,33.49,NASDAQ,45.75,26.580750,-0.073404,0.005027,H,2338.047550,S
3701853,ZYNE,2022-12-23 00:00:00,0.539900,0.999815,0.009912,2022Q3,33.49,NASDAQ,45.75,24.705000,-0.091675,-0.000498,H,2315.115200,S
3701854,ZYNE,2022-12-27 00:00:00,0.500000,0.926097,-0.036237,2022Q3,33.49,NASDAQ,45.75,24.700425,-0.085593,0.000889,M,2324.022300,S
3701855,ZYNE,2022-12-28 00:00:00,0.515700,1.031400,-0.161496,2022Q3,33.49,NASDAQ,45.75,22.875000,-0.070436,0.007913,L,2315.248000,S


In [ ]:
df.to_sql("df", conn, if_exists='replace')

query='''
        SELECT DISTINCT date, ticker, exchange, adj_close, mkt_cap_2, size_portfolio
        FROM df'''

df=pd.read_sql(query, conn)

In [ ]:
daily_returns=df

# CALCULATE MOMENTUM RETURNS

In [ ]:
 #PRIOR_RET is from - 250 to - 21.

In [ ]:
daily_returns

,date,ticker,exchange,adj_close,mkt_cap_2,size_portfolio
0,2018-02-05 00:00:00,A,NYSE,65.806086,22119.012687,B
1,2018-02-06 00:00:00,A,NYSE,66.027947,21178.372568,B
2,2018-02-07 00:00:00,A,NYSE,65.651747,21249.774294,B
3,2018-02-08 00:00:00,A,NYSE,62.748254,21128.701803,B
4,2018-02-09 00:00:00,A,NYSE,64.609962,20194.270530,B
...,...,...,...,...,...,...
3701103,2022-12-22 00:00:00,ZYNE,NASDAQ,0.540000,26.580750,S
3701104,2022-12-23 00:00:00,ZYNE,NASDAQ,0.539900,24.705000,S
3701105,2022-12-27 00:00:00,ZYNE,NASDAQ,0.500000,24.700425,S
3701106,2022-12-28 00:00:00,ZYNE,NASDAQ,0.515700,22.875000,S


In [ ]:
daily_returns=daily_returns.sort_values(by=["ticker", "date"])

daily_returns['date']=pd.to_datetime(daily_returns['date'])

In [ ]:
daily_returns['daily_return']=daily_returns.groupby('ticker')['adj_close'].pct_change() +1

In [ ]:
for i in range(21, 251):
    daily_returns[f'daily_return_shifted_{i}'] = daily_returns.groupby('ticker')['daily_return'].shift(i)

<ipython-input-11-e815257fab9e>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_returns[f'daily_return_shifted_{i}'] = daily_returns.groupby('ticker')['daily_return'].shift(i)


In [ ]:
daily_returns.dropna(inplace=True)

In [ ]:
cols_to_exclude = ['date', 'ticker', 'adj_close', 'mkt_cap_2', 'size_portfolio', 'daily_return']
cols = [col for col in daily_returns.columns if col not in cols_to_exclude]
cols

In [ ]:
daily_returns['prior_month_ret'] = daily_returns[cols].prod(axis=1)

<ipython-input-14-d4aec157acd8>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  daily_returns['prior_month_ret'] = daily_returns[cols].prod(axis=1)


In [ ]:
for i in range(21, 251):
  daily_returns=daily_returns.drop(columns=[f'daily_return_shifted_{i}'])

In [ ]:
file="daily_returns_momentum.csv"

daily_returns.to_csv(filepath + file)

# DAILY RETURN SORTS

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps
from sqlite3 import connect
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

conn=connect(':memory:')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=e85d75997c88ca8f1bb008d4f52f7a47fc78e5f2d137ec007015e89ea7a1d596
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


In [ ]:
filepath="/content/gdrive/MyDrive/"
file="daily_returns_momentum.csv"
daily_returns=pd.read_csv(filepath + file)
daily_returns

In [ ]:
daily_returns

In [ ]:
#filter for relevant exchanges
daily_returns=daily_returns[(daily_returns['exchange']=="NYSE") | (daily_returns['exchange']=="AMEX") | (daily_returns['exchange']=="NASDAQ")]
daily_returns['date']=pd.to_datetime(daily_returns['date'])

In [ ]:
daily_returns_sorts= daily_returns.dropna(subset=['prior_month_ret', 'mkt_cap_2'])

In [ ]:
daily_returns_sorts.sort_values(by='date')

,index,Unnamed: 0,date,ticker,exchange,adj_close,mkt_cap_2,size_portfolio,daily_return,prior_month_ret,return_q30,return_q70,return_portfolio
1277731,1277731,1755781,2018-08-06 00:00:00,JEF,NYSE,20.057379,6910.569365,B,1.000000,0.874687,0.873613,0.874227,H
1277730,1277730,1755780,2018-08-06 00:00:00,JEF,NYSE,20.057379,6933.199146,B,0.996736,0.873152,0.873613,0.874227,L
1277732,1277732,1755782,2018-08-07 00:00:00,JEF,NYSE,20.024538,6910.569365,B,0.998363,0.928362,0.931538,0.935772,L
1277733,1277733,1755783,2018-08-07 00:00:00,JEF,NYSE,20.024538,6899.254474,B,1.000000,0.938948,0.931538,0.935772,H
1277734,1277734,1755784,2018-08-08 00:00:00,JEF,NYSE,19.991698,6899.254474,B,0.998360,0.940467,0.934186,0.937775,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
926997,926997,1270342,2022-12-28 00:00:00,FONR,NASDAQ,17.000000,110.695000,S,1.005917,1.266758,0.653468,1.020183,H
2304517,2304517,3154842,2022-12-28 00:00:00,SUM,NYSE,28.460000,3435.097400,B,0.980703,0.732200,0.653468,1.020183,M
95887,95887,131990,2022-12-28 00:00:00,AKYA,NASDAQ,8.600000,340.501200,S,0.958751,0.917595,0.653468,1.020183,M
1038417,1038417,1426419,2022-12-28 00:00:00,GPOR,NYSE,72.940000,1431.953700,S,0.981564,1.034166,0.653468,1.020183,H


In [ ]:
#filter for NYSE stocks only
return_breakpoints=daily_returns_sorts[daily_returns['exchange']=="NYSE"]

q30_ret=return_breakpoints.groupby('date').apply(lambda x: np.quantile(x.prior_month_ret, 0.3))
q70_ret=return_breakpoints.groupby('date').apply(lambda x: np.quantile(x.prior_month_ret, 0.7))

q30_ret=pd.DataFrame(q30_ret)
q30_ret=q30_ret.rename(columns={0: 'q30'})

q70_ret=pd.DataFrame(q70_ret)
q70_ret=q70_ret.rename(columns={0: 'q70'})

In [ ]:
return_quantiles=pd.merge(q30_ret, q70_ret, on='date')

In [ ]:
daily_returns

daily_returns_sorts.to_sql("dr", conn, if_exists='replace')
return_quantiles.to_sql("q", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT dr.*, q.q30 as return_q30, q.q70 as return_q70
        FROM dr
        LEFT JOIN q
        ON dr.date=q.date'''

daily_returns_sorts_quantiles=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
daily_returns_sorts=daily_returns_sorts_quantiles

daily_returns_sorts['return_portfolio']=np.where(daily_returns_sorts['prior_month_ret'] > daily_returns_sorts['return_q70'], 'H', np.where(daily_returns_sorts['return_q30']> daily_returns_sorts['prior_month_ret'], 'L', 'M'))


In [ ]:
daily_returns_sorts

#PORTFOLIO CONSTRUCTION 

In [ ]:
daily_returns_portfolios=daily_returns_sorts

In [ ]:
#parse relevant variables

daily_returns_portfolios.to_sql('p', conn, if_exists='replace')

query=''' 
        SELECT DISTINCT p.ticker, p.date, p.adj_close, p.daily_return, p.prior_month_ret as prior_daily_ret, p.mkt_cap_2 as mkt_val, p.return_portfolio, p.size_portfolio
          FROM p
        '''

portfolio_construction=pd.read_sql(query, conn)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
portfolio_construction

In [ ]:
main=portfolio_construction

In [ ]:
main

,ticker,date,adj_close,daily_return,prior_daily_ret,mkt_val,return_portfolio,size_portfolio
0,A,2019-02-05 00:00:00,74.297811,1.003683,0.968511,23597.011137,M,B
1,A,2019-02-06 00:00:00,74.288075,0.999869,0.985753,23683.913191,M,B
2,A,2019-02-07 00:00:00,73.343652,0.987287,1.005935,23680.809546,M,B
3,A,2019-02-08 00:00:00,73.801259,1.006239,1.074516,23379.756004,M,B
4,A,2019-02-11 00:00:00,74.073876,1.003694,1.053349,23525.627308,M,B
...,...,...,...,...,...,...,...,...
2703519,ZYNE,2022-12-21 00:00:00,0.581000,1.019298,0.164486,26.077500,L,S
2703520,ZYNE,2022-12-22 00:00:00,0.540000,0.929432,0.164087,26.580750,L,S
2703521,ZYNE,2022-12-23 00:00:00,0.539900,0.999815,0.169209,24.705000,L,S
2703522,ZYNE,2022-12-27 00:00:00,0.500000,0.926097,0.183672,24.700425,L,S


In [ ]:
main['daily_return']=pd.to_numeric(main['daily_return'], errors='coerce')
main['mkt_val']=pd.to_numeric(main['mkt_val'], errors='coerce')

In [ ]:
main.dropna(inplace=True)
valueWeightRet=main.groupby(['date', 'size_portfolio', 'return_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['daily_return']), weights=pd.to_numeric(x['mkt_val'])))
valueWeightRet_df=pd.DataFrame(valueWeightRet)
valueWeightRet_df=valueWeightRet_df.reset_index(level=['size_portfolio', 'return_portfolio'])
valueWeightRet_df

,size_portfolio,return_portfolio,0
date,,,
2018-08-06 00:00:00,B,H,1.000000
2018-08-06 00:00:00,B,L,0.996736
2018-08-07 00:00:00,B,H,1.000000
2018-08-07 00:00:00,B,L,0.998363
2018-08-08 00:00:00,B,H,0.998360
...,...,...,...
2022-12-28 00:00:00,B,L,0.989358
2022-12-28 00:00:00,B,M,0.985384
2022-12-28 00:00:00,S,H,0.986684


In [ ]:
portfolios=valueWeightRet_df
portfolios[0]=portfolios[0]-1
portfolios[0]*=100
portfolios

,size_portfolio,return_portfolio,0
date,,,
2018-08-06 00:00:00,B,H,0.000000
2018-08-06 00:00:00,B,L,-0.326397
2018-08-07 00:00:00,B,H,0.000000
2018-08-07 00:00:00,B,L,-0.163733
2018-08-08 00:00:00,B,H,-0.164002
...,...,...,...
2022-12-28 00:00:00,B,L,-1.064151
2022-12-28 00:00:00,B,M,-1.461579
2022-12-28 00:00:00,S,H,-1.331640


In [ ]:
portfolios['date']=portfolios.index

In [ ]:
portfolios['date']=pd.to_datetime(portfolios['date'])

In [ ]:
portfolios=portfolios[portfolios['date']>"2019-01-01"]

In [ ]:
portfolios.drop(columns='date', inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
x=['size_portfolio', 'return_portfolio']

for x in x:
  portfolios[x]=portfolios[x].apply(str)

portfolios['size_return_portfolio']=portfolios['size_portfolio'] + "/" + portfolios['return_portfolio'] 

portfolios=portfolios.drop(['size_portfolio', 'return_portfolio'], axis=1)

portfolios=pd.pivot(portfolios, columns='size_return_portfolio', values=0)

<ipython-input-91-48201ee453bf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolios[x]=portfolios[x].apply(str)
<ipython-input-91-48201ee453bf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolios['size_return_portfolio']=portfolios['size_portfolio'] + "/" + portfolios['return_portfolio']


In [ ]:
portfolios.dropna(inplace=True)

In [ ]:
portfolios

size_return_portfolio,B/H,B/L,B/M,S/H,S/L,S/M
date,,,,,,
2019-02-04 00:00:00,0.792184,1.041999,0.910714,0.980330,0.919678,0.924638
2019-02-05 00:00:00,0.517714,0.966927,0.752271,0.511611,0.270071,0.264588
2019-02-06 00:00:00,-0.265139,-0.174542,-0.310716,-0.642572,0.262699,-0.003028
2019-02-07 00:00:00,-0.803907,-1.448362,-1.053299,-1.046211,-2.323964,-1.197962
2019-02-08 00:00:00,0.354908,0.306944,0.027870,0.773036,-0.501898,0.269463
...,...,...,...,...,...,...
2022-12-21 00:00:00,1.243762,1.558900,1.512604,1.487015,1.413111,1.329821
2022-12-22 00:00:00,-0.838011,-2.906827,-1.538273,-1.136540,-1.439268,-1.126709
2022-12-23 00:00:00,0.675173,0.150575,0.455173,0.739654,-0.542414,0.070112


In [ ]:
portfolios
x=['B/H', 'B/L', 'B/M', 'S/H', 'S/L', 'S/M']
for x in x:
  portfolios[x]=pd.to_numeric(portfolios[x], errors='coerce')


In [ ]:
portfolios['MOM']= 0.5 * (portfolios['S/H'] + portfolios['B/H']) - 0.5 * (portfolios['S/L'] + portfolios['B/L'])

# IMPORT FAMA FRENCH

In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
datasets = get_available_datasets()
datasets

In [ ]:

FF_MOM_Factor=[dataset for dataset in datasets if 'F-F_Momentum_Factor_daily' in dataset ]
FF_MOM_Factor=web.DataReader(FF_MOM_Factor[0],'famafrench',start='2017-11-01',end='2023-01-01')[0]

In [ ]:
FF_MOM_Factor['Date']=FF_MOM_Factor.index
FF_MOM_Factor['Date']=FF_MOM_Factor['Date'].apply(str)
FF_MOM_Factor['Date']=pd.to_datetime(FF_MOM_Factor['Date'], format="%Y-%m")

FF_MOM_Factor=FF_MOM_Factor.set_index('Date', drop=True)
portfolios.to_sql('p', conn, if_exists='replace')
FF_MOM_Factor.to_sql('c', conn, if_exists='replace')

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
query=''' 
        SELECT DISTINCT p.MOM, c.*
        FROM p
        LEFT JOIN c ON p.date=c.Date
         '''

checker=pd.read_sql(query, conn)

In [ ]:
checker

,MOM,Date,Mom
0,-0.094582,2019-02-04 00:00:00,-0.15
1,-0.103836,2019-02-05 00:00:00,0.10
2,-0.497934,2019-02-06 00:00:00,-0.29
3,0.961104,2019-02-07 00:00:00,0.69
4,0.661449,2019-02-08 00:00:00,0.73
...,...,...,...
979,-0.120617,None,NaN
980,1.185772,None,NaN
981,0.903333,None,NaN
982,2.313570,None,NaN


In [ ]:
checker_clean=checker.dropna()

In [ ]:
checker_clean.set_index('Date', inplace=True)

In [ ]:
checker_clean.corr(method='pearson')

,MOM,Mom
MOM,1.000000,0.978947
Mom,0.978947,1.000000


In [ ]:
portfolios

size_return_portfolio,B/H,B/L,B/M,S/H,S/L,S/M,MOM
date,,,,,,,
2019-02-04 00:00:00,0.792184,1.041999,0.910714,0.980330,0.919678,0.924638,-0.094582
2019-02-05 00:00:00,0.517714,0.966927,0.752271,0.511611,0.270071,0.264588,-0.103836
2019-02-06 00:00:00,-0.265139,-0.174542,-0.310716,-0.642572,0.262699,-0.003028,-0.497934
2019-02-07 00:00:00,-0.803907,-1.448362,-1.053299,-1.046211,-2.323964,-1.197962,0.961104
2019-02-08 00:00:00,0.354908,0.306944,0.027870,0.773036,-0.501898,0.269463,0.661449
...,...,...,...,...,...,...,...
2022-12-21 00:00:00,1.243762,1.558900,1.512604,1.487015,1.413111,1.329821,-0.120617
2022-12-22 00:00:00,-0.838011,-2.906827,-1.538273,-1.136540,-1.439268,-1.126709,1.185772
2022-12-23 00:00:00,0.675173,0.150575,0.455173,0.739654,-0.542414,0.070112,0.903333


In [ ]:
columns_to_keep =  ['MOM']
df = portfolios.loc[:, columns_to_keep]

In [ ]:
df.to_csv(filepath + "cleaned ff daily momentum factors.csv")